# Análisis de Compatibilidad de Datos CMIP6 y CR2MET
## Valle de Aconcagua, Chile

Pipeline para bias correction usando Quantile Mapping. Empezamos con un análisis sistemático de los datasets para entender su estructura y compatibilidad.

In [1]:
# Imports básicos para análisis de datos
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports completados")
print(f"✓ xarray version: {xr.__version__}")
print(f"✓ numpy version: {np.__version__}")
print(f"✓ pandas version: {pd.__version__}")

✓ Imports completados
✓ xarray version: 2025.1.2
✓ numpy version: 2.2.3
✓ pandas version: 2.2.3


In [2]:
# Configuración de paths
BASE_PATH = Path("/home/aninotna/magister/tesis/justh2_pipeline")
DATA_PATH = BASE_PATH / "data"
CMIP6_PATH = DATA_PATH / "cmip6" / "historical"
CR2MET_PATH = DATA_PATH / "cr2met" / "clima.zarr"

# Región Valle de Aconcagua
BBOX = {
    'lat_min': -33.27,
    'lat_max': -32.26, 
    'lon_min': -71.89,
    'lon_max': -70.00
}

print("✓ Configuración completada")
print(f"✓ CMIP6 path: {CMIP6_PATH}")
print(f"✓ CR2MET path: {CR2MET_PATH}")
print(f"✓ Región Valle de Aconcagua: {BBOX}")

✓ Configuración completada
✓ CMIP6 path: /home/aninotna/magister/tesis/justh2_pipeline/data/cmip6/historical
✓ CR2MET path: /home/aninotna/magister/tesis/justh2_pipeline/data/cr2met/clima.zarr
✓ Región Valle de Aconcagua: {'lat_min': -33.27, 'lat_max': -32.26, 'lon_min': -71.89, 'lon_max': -70.0}


In [ ]:
# 1. ANÁLISIS DE CR2MET
print("🔍 ANALIZANDO DATASET CR2MET")
print("=" * 40)

try:
    # Cargar CR2MET
    cr2met = xr.open_dataset(CR2MET_PATH)
    print(f"✓ CR2MET cargado exitosamente")
    
    # Información básica
    print(f"\n📊 INFORMACIÓN GENERAL:")
    print(f"  Variables: {list(cr2met.data_vars)}")
    print(f"  Dimensiones: {dict(cr2met.dims)}")
    print(f"  Coordenadas: {list(cr2met.coords)}")
    
    # Información espacial
    print(f"\n🗺️ INFORMACIÓN ESPACIAL:")
    print(f"  Latitud: {cr2met.lat.values.min():.3f} a {cr2met.lat.values.max():.3f}")
    print(f"  Longitud: {cr2met.lon.values.min():.3f} a {cr2met.lon.values.max():.3f}")
    print(f"  Resolución lat: ~{np.diff(cr2met.lat.values)[0]:.3f}°")
    print(f"  Resolución lon: ~{np.diff(cr2met.lon.values)[0]:.3f}°")
    
    # Información temporal
    print(f"\n⏰ INFORMACIÓN TEMPORAL:")
    print(f"  Período: {cr2met.time.values[0]} a {cr2met.time.values[-1]}")
    print(f"  Total días: {len(cr2met.time)}")
    time_diff = pd.to_datetime(cr2met.time.values[1]) - pd.to_datetime(cr2met.time.values[0])
    print(f"  Frecuencia: {time_diff.days} día(s)")
    
    # Análisis de precipitación
    if 'pr' in cr2met.data_vars:
        pr = cr2met['pr']
        print(f"\n🌧️ PRECIPITACIÓN (PR):")
        print(f"  Shape: {pr.shape}")
        print(f"  Unidades: {pr.attrs.get('units', 'No especificadas')}")
        print(f"  Min: {float(pr.min().values):.3f}")
        print(f"  Max: {float(pr.max().values):.3f}")
        print(f"  Media: {float(pr.mean().values):.3f}")
        
        # Recorte al Valle de Aconcagua
        pr_valle = pr.sel(
            lat=slice(BBOX['lat_min'], BBOX['lat_max']),
            lon=slice(BBOX['lon_min'], BBOX['lon_max'])
        )
        print(f"\n✂️ RECORTE VALLE DE ACONCAGUA:")
        print(f"  Shape: {pr_valle.shape}")
        print(f"  Rango lat: {pr_valle.lat.values.min():.3f} a {pr_valle.lat.values.max():.3f}")
        print(f"  Rango lon: {pr_valle.lon.values.min():.3f} a {pr_valle.lon.values.max():.3f}")
        print(f"  Media regional: {float(pr_valle.mean().values):.3f}")
        
        # Guardar para análisis posterior
        cr2met_pr_valle = pr_valle
        
    else:
        print(f"\n❌ Variable 'pr' no encontrada en CR2MET")
        
except Exception as e:
    print(f"❌ Error cargando CR2MET: {e}")
    
print(f"\n✅ Análisis CR2MET completado")

🔍 ANALIZANDO DATASET CR2MET
✓ CR2MET cargado exitosamente

📊 INFORMACIÓN GENERAL:
  Variables: ['year', 'cl_mask', 'pr', 'tmin', 'pr_sd', 'tmax']
  Dimensiones: {'time': 22646, 'lat': 800, 'lon': 220}
  Coordenadas: ['lat', 'time', 'lon']

🗺️ INFORMACIÓN ESPACIAL:
  Latitud: -56.975 a -17.025
  Longitud: -76.975 a -66.025
  Resolución lat: ~0.050°
  Resolución lon: ~0.050°

⏰ INFORMACIÓN TEMPORAL:
  Período: 1960-01-01T00:00:00.000000000 a 2021-12-31T00:00:00.000000000
  Total días: 22646
  Frecuencia: 1 día(s)

🌧️ PRECIPITACIÓN (PR):
  Shape: (22646, 800, 220)
  Unidades: mm/day


In [ ]:
# 2. ANÁLISIS DE ARCHIVOS CMIP6
print("🔍 ANALIZANDO ARCHIVOS CMIP6")
print("=" * 40)

# Listar archivos disponibles
pr_path = CMIP6_PATH / "pr"
if pr_path.exists():
    archivos = list(pr_path.glob("*.nc"))
    print(f"📁 Archivos encontrados: {len(archivos)}")
    
    for archivo in sorted(archivos):
        print(f"  - {archivo.name} ({archivo.stat().st_size / (1024**3):.2f} GB)")
        
    # Analizar primer archivo como ejemplo
    if archivos:
        archivo_ejemplo = archivos[0]
        print(f"\n🔬 ANÁLISIS DETALLADO: {archivo_ejemplo.name}")
        
        try:
            # Intentar cargar con diferentes engines
            for engine in ['netcdf4', 'h5netcdf', 'scipy']:
                try:
                    ds = xr.open_dataset(archivo_ejemplo, engine=engine)
                    print(f"  ✓ Archivo cargado exitosamente con engine: {engine}")
                    break
                except Exception as e:
                    print(f"  ❌ Engine {engine} falló: {e}")
                    continue
            
            if 'ds' in locals():
                # Información general
                print(f"\n📊 INFORMACIÓN GENERAL:")
                print(f"  Variables: {list(ds.data_vars)}")
                print(f"  Dimensiones: {dict(ds.dims)}")
                print(f"  Coordenadas: {list(ds.coords)}")
                
                # Información espacial
                print(f"\n🗺️ INFORMACIÓN ESPACIAL:")
                print(f"  Latitud: {ds.lat.values.min():.3f} a {ds.lat.values.max():.3f}")
                print(f"  Longitud: {ds.lon.values.min():.3f} a {ds.lon.values.max():.3f}")
                
                # Verificar si necesita conversión de coordenadas (0-360 -> -180-180)
                if ds.lon.values.max() > 180:
                    print(f"  ⚠️ Coordenadas en formato 0-360°, necesita conversión a -180-180°")
                else:
                    print(f"  ✓ Coordenadas ya en formato -180-180°")
                
                # Información temporal
                print(f"\n⏰ INFORMACIÓN TEMPORAL:")
                print(f"  Período: {ds.time.values[0]} a {ds.time.values[-1]}")
                print(f"  Total timesteps: {len(ds.time)}")
                if len(ds.time) > 1:
                    time_diff = pd.to_datetime(ds.time.values[1]) - pd.to_datetime(ds.time.values[0])
                    print(f"  Frecuencia: {time_diff.days} día(s)")
                
                # Análisis de precipitación
                if 'pr' in ds.data_vars:
                    pr_cmip = ds['pr']
                    print(f"\n🌧️ PRECIPITACIÓN (PR):")
                    print(f"  Shape: {pr_cmip.shape}")
                    print(f"  Unidades: {pr_cmip.attrs.get('units', 'No especificadas')}")
                    print(f"  Dtype: {pr_cmip.dtype}")
                    
                    # Estadísticos básicos (muestra pequeña para evitar problemas de memoria)
                    sample = pr_cmip.isel(time=slice(0, 10))
                    print(f"  Min (muestra): {float(sample.min().values):.6f}")
                    print(f"  Max (muestra): {float(sample.max().values):.6f}")
                    print(f"  Media (muestra): {float(sample.mean().values):.6f}")
                    
                    # Verificar traslape espacial con Valle de Aconcagua
                    lat_overlap = (ds.lat.values >= BBOX['lat_min']) & (ds.lat.values <= BBOX['lat_max'])
                    
                    # Ajustar longitudes si es necesario
                    if ds.lon.values.max() > 180:
                        lon_adjusted = ((ds.lon.values + 180) % 360) - 180
                        lon_overlap = (lon_adjusted >= BBOX['lon_min']) & (lon_adjusted <= BBOX['lon_max'])
                    else:
                        lon_overlap = (ds.lon.values >= BBOX['lon_min']) & (ds.lon.values <= BBOX['lon_max'])
                    
                    print(f"\n🎯 TRASLAPE CON VALLE DE ACONCAGUA:")
                    print(f"  Pixeles latitud en región: {lat_overlap.sum()}")
                    print(f"  Pixeles longitud en región: {lon_overlap.sum()}")
                    
                    if lat_overlap.sum() > 0 and lon_overlap.sum() > 0:
                        print(f"  ✓ Hay traslape espacial con la región")
                    else:
                        print(f"  ❌ No hay traslape espacial con la región")
                
                # Cerrar dataset
                ds.close()
                
        except Exception as e:
            print(f"  ❌ Error analizando archivo: {e}")
            
else:
    print(f"❌ Directorio {pr_path} no existe")
    
print(f"\n✅ Análisis CMIP6 completado")

In [ ]:
# 3. ANÁLISIS DE COMPATIBILIDAD TEMPORAL
print("🔍 ANÁLISIS DE COMPATIBILIDAD TEMPORAL")
print("=" * 45)

if 'cr2met' in locals() and 'ds' in locals():
    # Períodos temporales
    cr2_start = pd.to_datetime(cr2met.time.values[0])
    cr2_end = pd.to_datetime(cr2met.time.values[-1])
    
    # Reabrir dataset CMIP6 para análisis temporal
    archivo_ejemplo = list((CMIP6_PATH / "pr").glob("*.nc"))[0]
    ds_temp = xr.open_dataset(archivo_ejemplo, engine='netcdf4')
    
    cmip_start = pd.to_datetime(ds_temp.time.values[0])
    cmip_end = pd.to_datetime(ds_temp.time.values[-1])
    
    print(f"📅 PERÍODOS TEMPORALES:")
    print(f"  CR2MET: {cr2_start.strftime('%Y-%m-%d')} a {cr2_end.strftime('%Y-%m-%d')} ({(cr2_end - cr2_start).days} días)")
    print(f"  CMIP6:  {cmip_start.strftime('%Y-%m-%d')} a {cmip_end.strftime('%Y-%m-%d')} ({(cmip_end - cmip_start).days} días)")
    
    # Calcular traslape
    overlap_start = max(cr2_start, cmip_start)
    overlap_end = min(cr2_end, cmip_end)
    
    if overlap_start <= overlap_end:
        overlap_days = (overlap_end - overlap_start).days
        print(f"\n✅ TRASLAPE TEMPORAL:")
        print(f"  Período: {overlap_start.strftime('%Y-%m-%d')} a {overlap_end.strftime('%Y-%m-%d')}")
        print(f"  Duración: {overlap_days} días ({overlap_days/365.25:.1f} años)")
        
        # Evaluar idoneidad para bias correction
        if overlap_days >= 365 * 10:  # Al menos 10 años
            print(f"  ✓ Suficiente para bias correction (>10 años)")
        elif overlap_days >= 365 * 5:  # Al menos 5 años
            print(f"  ⚠️ Marginal para bias correction (5-10 años)")
        else:
            print(f"  ❌ Insuficiente para bias correction (<5 años)")
    else:
        print(f"\n❌ NO HAY TRASLAPE TEMPORAL")
    
    ds_temp.close()
    
else:
    print(f"❌ Datasets no cargados, no se puede analizar compatibilidad temporal")
    
print(f"\n✅ Análisis de compatibilidad temporal completado")

In [ ]:
# 4. RESUMEN Y RECOMENDACIONES
print("📋 RESUMEN Y RECOMENDACIONES PARA BIAS CORRECTION")
print("=" * 55)

print(f"\n🎯 DATASETS IDENTIFICADOS:")
print(f"  ✓ CR2MET: Datos de referencia observacional")
print(f"  ✓ CMIP6: Datos de modelo climático (ACCESS-CM2)")

print(f"\n⚙️ PASOS NECESARIOS PARA COMPATIBILIZACIÓN:")
print(f"  1. 🗺️ Regridding espacial: CMIP6 → CR2MET grid")
print(f"  2. ⏰ Alineación temporal: encontrar período de traslape")
print(f"  3. 🔧 Estandarización de unidades: kg/m²/s → mm/día")
print(f"  4. 📅 Alineación de calendarios: manejar años bisiestos")
print(f"  5. 🌧️ Wet-day adjustment: manejar días secos en precipitación")

print(f"\n🔧 MÉTODOS DE BIAS CORRECTION RECOMENDADOS:")
print(f"  • Empirical Quantile Mapping (EQM): Método estándar")
print(f"  • Detrended Quantile Mapping (DQM): Para preservar tendencias")

print(f"\n⚠️ CONSIDERACIONES IMPORTANTES:")
print(f"  • Resolución espacial muy diferente (CMIP6 vs CR2MET)")
print(f"  • Verificar conversión de coordenadas 0-360° → -180-180°")
print(f"  • Manejar chunks de dask para datasets grandes")
print(f"  • Validar resultados en período de entrenamiento")

print(f"\n🚀 PRÓXIMO PASO:")
print(f"  Implementar pipeline de compatibilización paso a paso")

print(f"\n✅ Análisis de compatibilidad completado")